In [ ]:
import pickle
import plotly.express as px
import numpy as np
import cv2
import json
from tqdm.notebook import tqdm
import einops
import matplotlib.pyplot as plt
from pathlib import Path
import os
import torch
from skimage.measure import find_contours
from visualization_utils.visualization import draw_on_image

from blender_cv_utils.projection_camera import apply_object_matrix_world, project_points

In [ ]:
import numpy as np

def add_visibility_to_skeleton(skeleton, mask):
    visibility = []
    for i in skeleton.round().astype(np.int32):
        for x, y, z in i:
            if x < 0 or y < 0 or x >= mask.shape[1] or y >= mask.shape[0]:
                visibility.append(-1)
            else:
                visibility.append(mask[y, x])
    visibility_array = np.array(visibility)
    visibility_reshaped = visibility_array.reshape(-1, 2)[..., None]
    skeleton_with_visibility = np.concatenate((skeleton, visibility_reshaped), axis=-1)
    return skeleton_with_visibility

In [ ]:
camera_name = 'Camera.001'

In [ ]:
path = 'C:/Users/ivand/Desktop/output_temp/default_sc1/'

global_params = json.load(open(os.path.join(path, 'global_params.json'), 'r'))

all_data = []
for annot_path in tqdm(sorted(Path(path).rglob('annots_frame*.json'))):
     n = int(annot_path.name.replace('annots_frame', '').replace('.json', ''))
     im = cv2.imread(os.path.join(path, f'{camera_name}_{n:04}.png'))
     data = json.load(
         open(os.path.join(path, f'annots_frame{n:04}.json'), 'r'))
     data[camera_name].update(global_params)

     objects_data = {}
     for index, i in enumerate(['Armature', 'Armature.001', 'Armature.002', 'Armature.003', 'Armature.004', ]):
            skeleton = []
            for name, bone in data[i]['pose'].items():
                if name in ['Bone.012.R.001', 'Bone.012.L.001', 'Bone.008.R.001', 'Bone.008.L.001', 'Bone.020']:  # IK bones
                    continue
                points = np.stack([np.array(bone['head']), np.array(bone['tail'])])
                points_transformed = apply_object_matrix_world(points, data[i]['matrix_world'])
                points = project_points(points_transformed, **data[camera_name]).numpy()
                dh = torch.dist(torch.tensor(np.array(data[camera_name]['matrix_world'])[:-1,3]), torch.tensor(points_transformed[0]), p=2).item()
                dt = torch.dist(torch.tensor(np.array(data[camera_name]['matrix_world'])[:-1,3]), torch.tensor(points_transformed[1]), p=2).item()
                points = np.concatenate((points, np.array([dh,dt])[...,None]), axis=1)
                skeleton.append(points)
            skeleton = np.stack(skeleton)
            
            mesh = data[data[i]['childrens'][0]]
            mask = plt.imread(os.path.join(path, mesh["name"], f'{camera_name}_{n:04}.png'))
            mask = (mask[:,:,0] > 0.01).astype(np.uint8)

            skeleton = add_visibility_to_skeleton(skeleton, mask)

        #     xraymask = plt.imread(os.path.join(path, mesh["name"], f'xray_{camera_name}_{n:04}.png'))[:, :, 3]
  
            im = draw_on_image(im, mask)
         
            contours = find_contours(mask.T, 0.5)
            contours = [np.array(i).round().astype(np.int32) for i in contours]
            contours = np.concatenate(contours)
            xmin, xmax = np.min(contours[:, 0]), np.max(contours[:, 0])
            ymin, ymax = np.min(contours[:, 1]), np.max(contours[:, 1])
            objects_data[i] = {
                'box':(xmin, ymin, xmax, ymax),
                'mask':mask[ymin:ymax, xmin:xmax],
                'sk': skeleton,  
                'impath':os.path.join(path, f'{camera_name}_{n:04}.png'),
            }

     all_data.append(objects_data)
    #  break

In [ ]:
pickle.dump(all_data, open(f'{path}/tempdata.pickle', 'wb'))

In [ ]:
import json
from visualization_utils.visualization import draw_on_image, skeleton_map_to_edges
edges = skeleton_map_to_edges(json.load(open('C:/Users/ivand/Desktop/output_temp/default_sc1/skeletons.json'))['Armature'],
                              excluded_bones=['Bone.012.R.001', 'Bone.012.L.001', 'Bone.008.R.001', 'Bone.008.L.001', 'Bone.020'])


In [ ]:
for N, sample in enumerate(all_data):

    objects_names = list(sample.keys())
    impath = sample[objects_names[0]]['impath']
    image = cv2.cvtColor(cv2.imread(impath), cv2.COLOR_BGR2RGB)

    h,w,_ = image.shape

    bboxes = []
    landmarks = []
    instances_number = len(sample)
    placeholder = np.zeros((instances_number,h,w), dtype=np.int32)

    for n, i in enumerate(sample.values()):
        xmin,ymin,xmax,ymax = i['box']
        bboxes.append((xmin,ymin,xmax,ymax, n)) # n is id of instance
        placeholder[n, ymin:ymax,xmin:xmax] = i['mask']
        landmarks.append(i['sk'])

    landmarks = np.concatenate(landmarks)
    landmarks = landmarks[:,0,:] # select only heads

    if np.any(landmarks[:,-1]==-1):
        landmarks[landmarks[:,-1]==-1, 0:2] = 0

    sample = {
        'image': image.astype(np.float32), 
        'keypoints': landmarks, 
        'masks': placeholder, 
        'bboxes': np.array(bboxes)}
    
    im = draw_on_image(image, skeletons=sample['keypoints'].reshape(-1, 34, 4), edges=edges)
    plt.imshow(im); plt.show()
    if N>10:
        break